In [1]:
%tensorflow_version 
from keras.datasets import imdb
from keras.preprocessing import  sequence
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
vocab_size = 88584

maxlen = 250
batch_size = 64

(train_data, train_labels),(test_data, test_labels) = imdb.load_data(num_words= vocab_size)

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
#Let's look at one review
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

# More Preprocessing

If we have a look at some of our loaded in reviews we'll notice that they are different lengths. This is an issue. We can not pass different length data into our neural network. Therefore we must make each review the same length.

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:

train_data = pad_sequences(train_data, maxlen)
test_data = pad_sequences(test_data, maxlen)  

In [ ]:
train_data[1]

# Creating The Model

Now it's time to create the model. we'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwords that feeds into a dense node to get our predicted sentiment.
32 stands for the output dimenstion of the vectors generated by the embedding layer. We can change this value .

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


# Training

In [27]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 62s 87ms/step - loss: 0.4537 - acc: 0.7831 - val_loss: 0.3488 - val_acc: 0.8522
Epoch 2/10
625/625 [==============================] - 20s 32ms/step - loss: 0.2654 - acc: 0.8975 - val_loss: 0.3575 - val_acc: 0.8694
Epoch 3/10
625/625 [==============================] - 14s 23ms/step - loss: 0.2042 - acc: 0.9252 - val_loss: 0.2931 - val_acc: 0.8774
Epoch 4/10
625/625 [==============================] - 11s 18ms/step - loss: 0.1678 - acc: 0.9411 - val_loss: 0.3011 - val_acc: 0.8836
Epoch 5/10
625/625 [==============================] - 10s 15ms/step - loss: 0.1354 - acc: 0.9538 - val_loss: 0.3188 - val_acc: 0.8776
Epoch 6/10
625/625 [==============================] - 9s 15ms/step - loss: 0.1176 - acc: 0.9619 - val_loss: 0.3329 - val_acc: 0.8684
Epoch 7/10
625/625 [==============================] - 8s 14ms/step - loss: 0.0989 - acc: 0.9679 - val_loss: 0.3364 - val_acc: 0.8752
Epoch 8/10
625/625 [==============================] - 9s 14ms/st

In [28]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.5739 - acc: 0.8348
[0.5739352703094482, 0.8348399996757507]


# Making Predictions

In [30]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return pad_sequences([tokens], maxlen)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [46]:
#While were  at it lets make a decore function
reverse_word_index = {value: key for (key, value) in word_index.items()}
for decode_integers(integers):

  PAD = 0
  text = ""
  for num in integers:
    if num!= PAD:
      text += reverse_word_index[num] + " "

  return text[:-1]

print(decode_integers(encoded))

SyntaxError: ignored

In [48]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1, 250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was so awesome! I really loved it and would watch it again bex=cause it was amazingly great"
predict(positive_review)

negative_review = "That movie sucked. I hate it and couldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 364ms/step
[0.41973287]
1/1 [==============================] - 0s 20ms/step
[0.06993384]
